In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
import random
import os

In [2]:
xtrain = np.load("../input/xtrain-data/traffic_xdata_train.npz")['arr_0']

In [3]:
xtrain.shape

In [4]:
ytrain = np.load("../input/ytrain-data/traffic_ydata_train.npy")

In [5]:
ytrain.shape

In [6]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [8]:
from tensorflow.keras.applications.vgg19 import VGG19  
with tpu_strategy.scope():
    vgg19 = VGG19(input_shape = [224,224,3], weights = 'imagenet', include_top = False)
    
    for layers in vgg19.layers:
        layers.trainable = False
        
    x = tf.keras.layers.Flatten()(vgg19.output)
    predictions = tf.keras.layers.Dense(58,activation = 'softmax')(x)
    model = tf.keras.Model(inputs = vgg19.input, outputs = predictions)
    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
model.summary()

In [9]:
history = model.fit(xtrain,ytrain,batch_size = 32,validation_split = 0.33,epochs = 100)

In [10]:
xtest = np.load("../input/xtest-data/traffic_xtest_train.npy")

In [11]:
xtest.shape

In [12]:
predictions = model.predict(xtest)

In [14]:
predictions[0]